In [13]:
import pandas as pd
import numpy as np
import sqlite3 as mysql
import time
import re
from datetime import datetime

In [14]:
conn = mysql.connect(r"D:\SQLBASE\weather.db")

In [15]:
cur = conn.cursor()

In [16]:
cur.execute('select count(ymd) from history_tem_right_20110101_20181225')
total_rows = cur.fetchone()[0]

In [17]:
conn2 = mysql.connect(r"D:\SQLBASE\WeatherBefore.db")
curs = conn2.cursor()

In [6]:
cur.execute('select * from history_tem_right_20110101_20181225')
col_name_list = [tuple[0] for tuple in cur.description]
for index in range(1, total_rows+1):
    million_piece = cur.fetchmany(1000000)
    if index < 1000000:
        continue
    one_piece_dict = dict(zip(col_name_list, one_piece))
    if one_piece_dict['city'] is not None:
        one_piece_dict['city_ymd'] = one_piece_dict['city'] + '-' + str(one_piece_dict['ymd'])
        ymd_time = datetime.strptime(str(one_piece_dict['ymd']), '%Y%m%d')
        one_piece_dict['year'] = str(one_piece_dict['year'])
        one_piece_dict['month'] = str(one_piece_dict['ymd'])[4:-2]
        one_piece_dict['day'] = str(one_piece_dict['ymd'])[-2:]
        try:
            one_piece_dict['ave_tm'] = float(one_piece_dict['ave_tm'])
        except:
            one_piece_dict['ave_tm'] = np.NaN
        one_piece_dict['weekday'] = ymd_time.isoweekday()
        one_piece_df = pd.DataFrame(one_piece_dict, index=[0])
        try:
            print(index, '正在写入\n')
            one_piece_df.to_sql(
                name='WeatherAll',
                con=conn2,
                chunksize=1000000,
                if_exists = 'append',
                index=False,
                )
        except mysql.IntegrityError as e:
            print('数据已经写过，跳过\n', e)
            continue

In [18]:
million_piece = cur.fetchmany(1000000)

In [19]:
million_piece

[]

In [0]:
conn2.commit()

In [0]:
conn2.close()

In [0]:
del all_data